In [18]:
import onnxruntime
import numpy as np
import cv2

In [20]:
def cv2_load(image_path):
    im = cv2.imread(image_path).astype(np.float32)
    assert im is not None, f'Image Not Found {image_path}'
    h0, w0 = im.shape[:2] # 1080, 1920 
    r = 640 / max(h0, w0)  # ratio # 640 / 1920
    interp = cv2.INTER_LINEAR # if (r > 1) else cv2.INTER_AREA
    im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
    img, ratio, pad = letterbox(im, [384, 640], auto=False, scaleup=False)
    img = img.transpose((2, 0, 1))[::-1]
    img = np.ascontiguousarray(img)
    img = img / 255.0
    return img[None, :]

def letterbox(im, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, ratio, (dw, dh)


In [21]:
def create_onnx_session(modelpath):
    options = None
    # # create and run session
    session = onnxruntime.InferenceSession(modelpath, sess_options=options, providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
    return session

def run_onnx_session(session, img):
    ort_inputs = {session.get_inputs()[0].name: img}

    prediction = session.run(None, ort_inputs)
    return prediction

In [22]:
onnx_session_a = create_onnx_session("/Users/maheshkumar/Downloads/onnxmodel")
# onnx_session_b = create_onnx_session("../MergedModels/tycho_yolo_v6.1.0/onnxmodel")
onnx_session_b = create_onnx_session("/Users/maheshkumar/Downloads/onnxmodel")
# onnx_session_b = create_onnx_session('../FeatureHeadsModels/tycho_yolo_v6.0.2_beta_v3/mhmodel_NO_NMS_2023_07_11_08_13.onnx')
# onnx_session_b = create_onnx_session('../MergedModels/tycho_yolo_v6.0.3/mhmodel_NO_NMS_2023_08_28_19_45_v5_1.onnx')

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [69]:

# onnx_session_b = create_onnx_session("../MergedModels/tycho_yolo_v6.1.0/onnxmodel")
# onnx_session_b = create_onnx_session('../FeatureHeadsModels/tycho_yolo_v6.0.2_beta_v3/mhmodel_NO_NMS_2023_07_11_08_13.onnx')

In [23]:
image_name = "/Users/maheshkumar/Desktop/test_image.png"

im = cv2_load(image_name)

In [24]:
onnx_prediction_a = run_onnx_session(onnx_session_a, im)
onnx_prediction_b = run_onnx_session(onnx_session_b, im)

In [25]:
for i in range(11):
    # print(i, onnx_prediction_a[i].shape, onnx_prediction_b[i].shape)
    if i > 1:
        print(i, onnx_prediction_a[i].shape, onnx_prediction_b[i].shape)
        np.testing.assert_almost_equal(onnx_prediction_a[i], onnx_prediction_b[i], decimal=3)

2 (1, 2) (1, 2)
3 (1, 2) (1, 2)
4 (1, 4) (1, 4)
5 (1, 5) (1, 5)
6 (1, 7) (1, 7)
7 (1, 6) (1, 6)
8 (1, 6) (1, 6)
9 (1, 6) (1, 6)
10 (1, 4) (1, 4)


In [26]:
onnx_pred = onnx_prediction_b[12][0]
cv2.imwrite("/Users/maheshkumar/code/segmentation_output2.png",onnx_pred.astype('uint8')*255)


True

In [16]:
onnx_pred.sum()

np.int64(1269)